# EWMA risk and volatility analysis

In [1]:
import sys  # no installation needed
from pathlib import Path  # no installation needed

ROOT = Path(r"C:\\Users\\quantbase\\Desktop\\ecom_forecast")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [2]:
import pandas as pd  # already in env - no new install
import matplotlib.pyplot as plt  # already in env - no new install

from src.config import ProjectPaths  # no installation needed
from src.risk import build_risk_table, flag_anomalies  # no installation needed


In [3]:
paths = ProjectPaths.from_root()
paths.ensure_directories()
risk_dir = paths.outputs_dir / 'risk'
risk_dir.mkdir(parents=True, exist_ok=True)
assumptions = paths.load_assumptions()
epsilon = float(assumptions.get('epsilon', 1e-9))
lam = float(assumptions.get('risk_regression', {}).get('ewma_lambda', 0.97))


In [4]:
driver_path = paths.outputs_dir / 'drivers' / 'driver_matrix.pkl'
if driver_path.exists():
    driver_matrix = pd.read_pickle(driver_path)
else:
    driver_matrix = pd.read_csv(paths.outputs_dir / 'drivers' / 'driver_matrix.csv')
if 'returns_abs' not in driver_matrix.columns and 'Returns' in driver_matrix.columns:
    driver_matrix['returns_abs'] = (-driver_matrix['Returns']).clip(lower=0)


In [5]:
risk_table = build_risk_table(driver_df=driver_matrix, lam=lam, epsilon=epsilon)
anomalies = flag_anomalies(risk_table, z_thresh=3.0)
assert len(risk_table) == len(driver_matrix)
assert not risk_table['Day'].duplicated().any()


In [6]:
cols = ['logret_Net sales', 'logret_Ad_Spend', 'logret_returns_abs', 'diff_cm']
corr = risk_table[cols].corr()
cov = risk_table[cols].cov()
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(corr, cmap='RdBu', vmin=-1, vmax=1)
ax.set_xticks(range(len(cols)))
ax.set_xticklabels(cols, rotation=45, ha='right')
ax.set_yticks(range(len(cols)))
ax.set_yticklabels(cols)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(risk_dir / 'corr_heatmap.png', dpi=200)
plt.close(fig)


In [7]:
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(cov, cmap='viridis')
ax.set_xticks(range(len(cols)))
ax.set_xticklabels(cols, rotation=45, ha='right')
ax.set_yticks(range(len(cols)))
ax.set_yticklabels(cols)
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(risk_dir / 'cov_heatmap.png', dpi=200)
plt.close(fig)

In [8]:
risk_table.to_csv(risk_dir / 'risk_table.csv', index=False)
anomalies.to_csv(risk_dir / 'anomaly_days.csv', index=False)
print('anomalies >=3:', len(anomalies))
if 'logret_returns_abs_z' in risk_table.columns:
    top_returns_abs = risk_table[['Day', 'logret_returns_abs_z']].copy()
    top_returns_abs['abs_z'] = top_returns_abs['logret_returns_abs_z'].abs()
    print(top_returns_abs.sort_values('abs_z', ascending=False).head(10))
['risk_table.csv', 'anomaly_days.csv', 'corr_heatmap.png', 'cov_heatmap.png']


anomalies >=3: 14
          Day  logret_returns_abs_z     abs_z
2  2025-09-19             -5.600298  5.600298
26 2025-10-13             -5.536114  5.536114
3  2025-09-20             -4.720186  4.720186
5  2025-09-22              4.280681  4.280681
27 2025-10-14              4.247529  4.247529
6  2025-09-23              3.962709  3.962709
25 2025-10-12             -3.723505  3.723505
4  2025-09-21             -3.318103  3.318103
19 2025-10-06              3.244602  3.244602
10 2025-09-27             -2.968410  2.968410


['risk_table.csv', 'anomaly_days.csv', 'corr_heatmap.png', 'cov_heatmap.png']

In [9]:
risk_table.shape

(91, 18)

In [10]:
anomalies.shape, anomalies.head(20), anomalies[anomalies["Day"] == "2025-09-23"]

((14, 7),
           Day          which_series   z_value  Net sales  returns_abs  \
 0  2025-09-19    logret_Net sales_z  5.600298   37746.52      7877.38   
 1  2025-09-19     logret_Ad_Spend_z  5.600298   37746.52      7877.38   
 2  2025-09-19  logret_returns_abs_z -5.600298   37746.52      7877.38   
 3  2025-09-20    logret_Net sales_z  4.203746   53603.15      4047.00   
 4  2025-09-20  logret_returns_abs_z -4.720186   53603.15      4047.00   
 5  2025-09-21  logret_returns_abs_z -3.318103   34462.50      2226.40   
 6  2025-09-22  logret_returns_abs_z  4.280681   27274.66      5851.90   
 7  2025-09-23     logret_Ad_Spend_z  5.303952   17362.43     21516.07   
 8  2025-09-23  logret_returns_abs_z  3.962709   17362.43     21516.07   
 9  2025-09-24    logret_Net sales_z  3.759993   39224.33     12233.12   
 10 2025-10-06  logret_returns_abs_z  3.244602   13504.70      8991.50   
 11 2025-10-12  logret_returns_abs_z -3.723505   19282.02       269.00   
 12 2025-10-13  logret_retur

In [11]:
risk_table.loc[risk_table['Day']=='2025-09-23', ['logret_returns_abs','logret_returns_abs_z','returns_abs','Net sales','Ad_Spend','CM$']]

,logret_returns_abs,logret_returns_abs_z,returns_abs,Net sales,Ad_Spend,CM$
6,1.302034,3.962709,21516.07,17362.43,15050,-6288.667818


In [12]:
# zscore similarity check

(risk_table["logret_Net sales_z"] - risk_table ["logret_Ad_Spend_z"]).abs().max()


6.743700128380529